In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import ase
from ase.neighborlist import natural_cutoffs, NeighborList
from ase.io.trajectory import Trajectory
from ase.io import read as ase_read

import sys
sys.path.append("../py_src/")

import hydrogen_tracing

import re
import pickle

In [ ]:
traj_path = '../test_data/221220_short_traj/anatase-100-nd-0/out.lammpstrj'
newrun = False

if newrun:
    p_file = open("chain_finding.dump", 'wb')
    trajectory = ase_read(traj_path, index='1000:10000:10') # index=':' loads the whole trajectory into memory, consider not doing that
    pickle.dump(trajectory, p_file)
else:
    p_file = open("chain_finding.dump", 'rb')
    trajectory = pickle.load(p_file)
# Could be better to just iterate through the file using index='%u'%ii

In [ ]:
print(len(trajectory))

In [ ]:
chain_finder = hydrogen_tracing.HOchainFinder(trajectory)
if newrun:
    config_dict, counts, config_codes = chain_finder.all_configs()
    pickle.dump([config_dict, counts, config_codes], p_file)
else:
    config_dict, counts, config_codes = pickle.load(p_file)

In [ ]:
configs = ['']*len(config_dict)
for key, value in config_dict.items():
    configs[value] = key

total_counts = np.sum(counts, axis=0)
print("In total found: ")
for ii_conf in range(total_counts.shape[0]):
    print(configs[ii_conf]+': ', total_counts[ii_conf])

In [ ]:
if newrun:
    special_list = chain_finder.find_special_configs()
    pickle.dump(special_list, p_file)
else:
    special_list = pickle.load(p_file)
p_file.close()

In [ ]:
import pandas as pd

symbols = []
for ii_special, specials in enumerate(special_list):
    ii_snapshot = ii_special + 1
    snapshot = chain_finder.trajectory[ii_snapshot]
    for special_config in specials:
        symbols.append(snapshot[special_config].get_chemical_formula(mode='hill'))
        
count = pd.Series(symbols).value_counts()
print(count)

In [ ]:
hop_list = []
for ii_step, specials in enumerate(special_list):
    # Go through all found special configurations
    ii_step += 1
    if ii_step % 100 == 0:
        print(ii_step)

    if len(special_list) != 0:
        for special_config in specials:
            hops = chain_finder.find_hopping(special_config[-1], ii_step, counter=0, verbose=False)
            hop_list.append(hops)
            if hops == 0:
                chain_finder.find_hopping(special_config[-1], ii_step, counter=0, verbose=True)

In [ ]:
hop_list = chain_finder.analyse_special_configs(special_list, verbose=False)

In [ ]:
%matplotlib inline
print(hop_list)
hops = np.asarray(hop_list)
print(np.min(hop_list), np.max(hop_list))
hop_counts = np.bincount(hop_list-np.min(hop_list))
print(hop_counts)

#-8, -7, -6, -5 ... -1
labels = ["Exchange of H?", "H ends at\nHOTi->H2O Ti", "H ends at\nHOTi->H2O Ti", "No change at site", "Surface Hopping", "Wrong Oxygen Observed", "H20 close to HOTi", "End of Chain", "OTi losing H"]
labels += np.arange(0, np.max(hop_list)+1).tolist()

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_title("Analysis of Ti hopping for H2O Splitting")
ax.set_xlabel('Error Codes \ Number of Hops')
ax.set_ylabel('# of occurences')

ax.bar(np.arange(np.min(hop_list), np.max(hop_list)+1), hop_counts)

ax.set_xticks(
    np.arange(np.min(hop_list), np.max(hop_list)+1),
    labels[-(np.max(hop_list) + 1 - np.min(hop_list)):],
    rotation=-45, ha="left"
)
plt.show()

In [ ]:
for key, value in config_dict.items():
    print(key, value)

In [ ]:
draw_codes = config_codes.copy()[1:, :]

water_bound = np.array([0, 3, 4], dtype=np.int8)
bulk_bound = np.array([1, 2], dtype=np.int8)

wb_tf = np.any(draw_codes[..., np.newaxis] == water_bound[np.newaxis, np.newaxis, :], axis=-1)

# Filter out oxygen that never changes state
has_changed = np.any(draw_codes != draw_codes[0, :], axis=0)
always_waterbound = np.all(np.any(draw_codes[..., np.newaxis] == water_bound[np.newaxis, np.newaxis, :], axis=-1), axis=0)
always_bulk = np.all(np.any(draw_codes[..., np.newaxis] == bulk_bound[np.newaxis, np.newaxis, :], axis=-1), axis=0)
always_relevant = np.logical_not(np.logical_or(always_waterbound, always_bulk))

particle_selection = np.logical_and(has_changed,always_relevant)

draw_codes = draw_codes[:, particle_selection]
draw_codes = draw_codes[:, :]

norm = mpl.colors.Normalize(vmin=np.min([v for v in config_dict.values()]), vmax=np.max([v for v in config_dict.values()]))
fig, ax = plt.subplots(1, 1, figsize=(25, 6))
im = ax.imshow(draw_codes.T, aspect='auto', norm=norm, cmap="tab10", interpolation='nearest')
ax.set_xlim([0, draw_codes.shape[0]-1])
cb = fig.colorbar(im)
cb.ax.set_yticks(np.arange(len(config_dict)), labels=[key for key in config_dict.keys()])
plt.show()

In [ ]:
%matplotlib auto

cur_step = 774
oxygen_index = 416
cur_gone_bound_oxygen = [65, 130, 64, 60, 70, 71]
neighs = [60, 63]

print(chain_finder.trajectory[cur_step][oxygen_index])
hydrogen_tracing.HOchainFinder.plot_snapshot(chain_finder.trajectory[cur_step], cur_gone_bound_oxygen[0], neighs)

In [ ]:
%matplotlib auto

plot_num = 1
print(n_specials[plot_num])
chain_finder.plot_special_config(trajectory[plot_num], natural_cutoffs(trajectory[plot_num], mult=0.75))